In [2]:
from Bio import SeqIO # pip3 install biopython
import pandas as pd
import glob
import os
import collections

In [20]:
def gb_to_fasta(path_of_files):
    df_cpgenome = pd.DataFrame()
    # genebank parsing & extract protein sequence
    for genbankfile in glob.glob(path_of_files):
        dic = {}
        IR  = []
        recs = [rec for rec in SeqIO.parse(genbankfile,'genbank')]
        for rec in recs:
            feats = [feat for feat in rec.features if feat.type == 'CDS']
            for feat in feats:
                try:
                    for gene in feat.qualifiers['gene']:
                        if gene in dic:
                            IR.append(gene)
                        else:
                            dic[gene] = str(feat.location.extract(parent_sequence = rec.seq).translate(table=11, to_stop=True))
                except KeyError:
                    pass
        # delete IR gene in dic       
        for IRgene in list(set(IR)):
            del dic[IRgene]
        # make dataframes
        species = genbankfile.split('/')[2].split('.')[0] 
        df = pd.DataFrame.from_dict(dic, orient = 'index', columns = [species])
        if len(df_cpgenome.index) == 0:
            df_cpgenome = df_cpgenome.append(df)
        else: # merge dataframes
            df_cpgenome = pd.merge(df_cpgenome,df, left_index=True, right_index=True)
    #return df_cpgenome
    # make fasta file each of cpgene
    for genename in list(df_cpgenome.index):
        ofile = open('./gene_fasta_11species/'+genename+'.fasta','w')
        for speciesname in df_cpgenome:
            ofile.write('>' + speciesname + '\n' + df_cpgenome[speciesname][genename] + '\n')

In [22]:
# prank.py
for fastafile in glob.glob('./gene_fasta_11species/*.fasta'):
    name = fastafile.split('/')[2].split('.')[0]
    os.system('prank -d='+fastafile+' -o=./gene_fasta_prank_11species/'+name+'.fasta -f=fasta')

In [23]:
# prottest.py -> terminal에서 돌려야 함

for file in glob.glob('./gene_fasta_prank_11species/*.nex'):
    #print(i)
    gene = file.split('/')[2].split('.')[0] 
    #print(gene)
    os.system('java -jar ../../../prottest-3.4.2/prottest-3.4.2.jar -i '+file+' -all-matrices -all-distributions -threads 2 > ./prottest_11species/'+gene+'.txt')

In [6]:
# prottest 결과 정리

prottest_list = []
for i in glob.glob('./prottest_11species/*.txt'):
    o = open(i)
    r = o.read()
    prottest_list.append(r.split('BIC: ')[1].split('\n')[0])
    prottest_list.append(r.split('LnL: ')[1].split('\n')[0])

b = collections.Counter(prottest_list).most_common()
b

[('CpREV+I+G', 13),
 ('CpREV', 13),
 ('CpREV+G', 12),
 ('CpREV+I', 11),
 ('HIVb+I+G', 9),
 ('HIVb+I', 7),
 ('HIVb+G', 6),
 ('JTT+G', 4),
 ('HIVb', 4),
 ('MtArt+I+G', 3),
 ('JTT+I', 3),
 ('MtArt', 3),
 ('MtArt+I', 3),
 ('HIVw+I+G', 3),
 ('HIVw+G', 3),
 ('JTT+I+G', 2),
 ('JTT', 2),
 ('FLU', 2),
 ('MtMam+I', 2),
 ('MtMam', 2),
 ('FLU+G', 1),
 ('FLU+I+G', 1),
 ('HIVw', 1),
 ('WAG+I', 1),
 ('WAG+I+G', 1)]